# 4 - Process rotary data

Loads adjusted data and formats for spectral analysis.

## Imports

Necessary modules for analysis.

In [1]:
# import modules

import xarray as xr
import numpy as np
import scipy.signal as sig
from scipy.stats import chi2
for i in range(2):
    %matplotlib notebook

In [2]:
# import data

adcp = 'Slope'     # Slope(2013,2014,2017,2018), Axis75(2013,2014), Axis55(2017,2018)
year = 2018
ds_in = xr.open_dataset(f'../Data/data/adj/adj_{adcp}_{year}_0.nc')

n_seg = ds_in.n_seg
if n_seg > 1:
    ds = [ds_in]
    for i in range(n_seg):
        if i > 0:
            ds_temp = xr.open_dataset(f'../Data/data/adj/adj_{adcp}_{year}_{i}.nc')
            ds.append(ds_temp)
elif n_seg == 1:
    ds = [ds_in]
    
# print(ds)

In [3]:
# extract variables

t_stamp = ds[0].t_stamp
depth = ds[0].depth.values
d = ds[0].d
start_date = ds[0].start_date
end_date = ds[0].end_date

## Format rotary data
Removes the mean from each depth. Performs rotary process with optimised parameters for visual analysis of spectra. Calculates 95% confidence intervals using a chi$^2$ method, the GM reference spectra is imported from gm.ipynb, WKB scaling is applied, and the noise floor is shown.

In [4]:
# spectra data adjustments & Welch parameters

time_total = 0
for i in range(n_seg):
    time_total += len(ds[i].t_seg)
print('Total time length:',time_total)

# set Welch parameters

fs = 1.11e-3                  # 4 samples per HOUR, 1.11e-3 per SECOND
win = 'hann'                  # optimal window for averaging
if time_total >= 20000:
    nps = 1024                # find optimal window for nperseg (1024 ~10 days)
elif time_total < 20000:
    nps = 512
overlap = nps // 2            # 50% overlap, default   

# remove short segments

t_short = []
for i in range(n_seg):
    if len(ds[i].t_seg) < nps:
        t_short.append(i)
for i in sorted(t_short, reverse=True):
    del ds[i]
n_seg = n_seg - len(t_short)

time_total = 0
for i in range(n_seg):
    time_total += len(ds[i].t_seg)
print(len(t_short),'segment(s) too short, new total time length:',time_total)

Total time length: 31363
0 segment(s) too short, new total time length: 31363


In [5]:
# remove mean at each depth

um,vm = [],[]
for i in range(n_seg):
    um_seg,vm_seg = [],[]
    uorig_temp = ds[i].uorig.values
    vorig_temp = ds[i].vorig.values
    for j in range(d):
        umtemp = np.copy(uorig_temp[:,j]) - np.nanmean(uorig_temp[:,j])
        vmtemp = np.copy(vorig_temp[:,j]) - np.nanmean(vorig_temp[:,j])
        um_seg.append(umtemp)
        vm_seg.append(vmtemp)
    um.append(um_seg)     # list[segment][depth][time]
    vm.append(vm_seg)     # 0 is upper depth index

In [6]:
# define rotary spectra function

def spec_rot(u, v):
    puf, pu = sig.welch(u,fs=fs,window=win,nperseg=nps,noverlap=overlap,return_onesided=False)  # auto-spectrum for u
    pvf, pv = sig.welch(v,fs=fs,window=win,nperseg=nps,noverlap=overlap,return_onesided=False)  # auto-spectrum for v
    cuvf, cuv = sig.csd(v,u,fs=fs,window=win,nperseg=nps,noverlap=overlap,return_onesided=False)# cross spectra (u,v --> v,u)
    quv = cuv.imag                                      # quadrature spectrum, imaginary part of cross spectra
    cw = ((pu + pv) - (2*quv)) / 2                      # rotatory components
    ccw = ((pu + pv) + (2*quv)) / 2
    F = puf                                             # frequency range (two-sided)
    
    return cuv, quv, cw, ccw, F

In [7]:
# rotary spectra at each depth

f_rot,cw_real,ccw_real = [],[],[]
for i in range(n_seg):
    cw_real_seg,ccw_real_seg = [],[]
    for j in range(d):
        cuv, quv, cw, ccw, f_rot_tot = spec_rot(um[i][j],vm[i][j])       # get rotary components (cw and ccw) and frequency range (f)
        half_idx = int((len(cw)/2))                            # discard half spectrum for real data 
        cw_real_seg.append(cw[0:half_idx]*2)                         # mult. amplitude by 2 to account for discarded data
        ccw_real_seg.append(ccw[0:half_idx]*2)     
    cw_real.append(cw_real_seg)
    ccw_real.append(ccw_real_seg)
f_rot.append(f_rot_tot[0:half_idx] ) # real frequency range (up to Nyquist)

In [8]:
# combine rotary segments and average with weighting, for each depth

time_weights = []
for i in range(n_seg):
    t_len = len(ds[i].t_seg)
    weight_temp = t_len/time_total
    time_weights.append(weight_temp)

cww_rot,ccww_rot = [],[]
for j in range(d):
    cww_rot_temp,ccww_rot_temp = [],[]
    for i in range(n_seg):
        if i == 0:
            cww_rot_temp.append(cw_real[i][j]*time_weights[i])
            ccww_rot_temp.append(ccw_real[i][j]*time_weights[i])
        elif i > 0:
            cww_rot_temp += (cw_real[i][j]*time_weights[i])
            ccww_rot_temp += (ccw_real[i][j]*time_weights[i])
    cww_rot.append(cww_rot_temp[0])      # list[depth][frequency]
    ccww_rot.append(ccww_rot_temp[0])

In [9]:
# apply WKB scaling at each depth

scaling_array = np.load('../project/archive/N2/scaling_array.npy')
GM_depths = scaling_array[0]                          # depths range from -4 to -924 metres
GM_scale = scaling_array[1]
int_scale = np.interp(depth,-GM_depths,GM_scale)

cw_rot_WKB,ccw_rot_WKB = [],[]

for i in range(d):                                    # values go in descending depth (0 is upper index)
    cw_rot_WKB.append(cww_rot[i]/int_scale[i])        # list[depth][frequency]
    ccw_rot_WKB.append(ccww_rot[i]/int_scale[i])

In [10]:
# error bars (95% confidence intervals) for each depth

probability = 0.95                            # calculate confidence intervals
alpha = 1 - probability        
NS = time_total / (nps / 2)                   # number of estimates, Welch
vp = (4/3)*NS                                 # for tapered windows
cp = chi2.ppf([1 - alpha / 2, alpha / 2], vp) # chi**2 distribution
cint = vp/cp                                  # interval coefficients

cw_conf_lower,cw_conf_upper,ccw_conf_lower,ccw_conf_upper = [],[],[],[]
for i in range(d):
    cw_conf_lower.append(cw_rot_WKB[i] * cint[0])               # define upper and lower confidence values
    cw_conf_upper.append(cw_rot_WKB[i] * cint[1])
    ccw_conf_lower.append(ccw_rot_WKB[i] * cint[0])             # define upper and lower confidence values
    ccw_conf_upper.append(ccw_rot_WKB[i] * cint[1])             # list[depth][frequency]

## Save
Save key values and arrays to NetCDF format using xarray.

In [11]:
# save to .nc files

ds_out = xr.Dataset( 
    data_vars=dict(
        cw_rot=(['depth','f_rot'], cw_rot_WKB),           # adjusted PSD data for each depth
        ccw_rot=(['depth','f_rot'], ccw_rot_WKB),
        cw_conf_lower=(['depth','f_rot'], cw_conf_lower),   # confidence intervals for PSD data at each depth
        cw_conf_upper=(['depth','f_rot'], cw_conf_upper),
        ccw_conf_lower=(['depth','f_rot'], ccw_conf_lower),
        ccw_conf_upper=(['depth','f_rot'], ccw_conf_upper),
    ),
    coords=dict(
        depth=depth,                 # depth values
        f_rot=f_rot[0],              # PSD frequency bins
    ),
    attrs=dict(
        description=f'Rotary data for {adcp} {t_stamp}.',
        adcp=adcp,                   # adcp
        t_stamp=t_stamp,             # time stamp
        t=time_total,                # length of initial time series
        d=d,                         # depth range
        start_date=start_date,       # start date of initial time series
        end_date=end_date,           # end date of initial time series
        fs=fs,                       # sampling frequency
        window=win,                  # fft window
        nps=nps,                     # fft segment length
        overlap=overlap,             # fft window overlap
    ),
) 

ds_out.to_netcdf(f'../Data/data/rot/rot_{adcp}_{t_stamp}.nc')